In [13]:
from keras.models import Sequential
from keras.layers import Lambda
from keras.layers import Dense, Flatten, Conv2D, Dropout, Cropping2D
from sklearn.model_selection import train_test_split
import pandas as pd
import shutil
import os
import sklearn.utils as utls
import imageio as imgio
import numpy as np
import cv2
import csv

path_img1 = 'data/IMG/'
path_data1 = 'data/driving_log.csv'
path_img2 = 'IMG/'
path_data2 = 'driving_log.csv'
#path_img = 'IMG1/'
#path_data = 'driving_log.csv'

def merge_driving_data(data, output_samples):
    with open(data) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            if line[0] == 'center' or line == '/home/workspace/CarND-Behavioral-Cloning-P3/IMG/center_2020_02_26_22_01_15_785.jpg' or line == '/home/workspace/CarND-Behavioral-Cloning-P3/IMG/left_2020_02_26_22_01_15_785.jpg' or line == '/home/workspace/CarND-Behavioral-Cloning-P3/IMG/right_2020_02_26_22_01_15_785.jpg':   
                continue
            output_samples.append(line)
    return output_samples

def combine_driving_data(driving_data1, driving_data2):
    combined_csv = pd.concat([pd.read_csv(driving_data1), pd.read_csv(driving_data2)])
    #export to csv
    combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')
    path_data = 'combined_csv.csv'
    return path_data

def copy_img(path1, path2): # Copy the images from one directory to the other
    for item in os.listdir(path2):
        s = os.path.join(path2, item)
        d = os.path.join(path1, item)
        if os.path.isdir(s):
            print(item)
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

def generator(samples, path_image, steering_correction, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        utls.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                name_center = path_image+'/'+batch_sample[0].split('/')[-1]
                name_left = path_image+'/'+batch_sample[1].split('/')[-1]
                name_right = path_image+'/'+batch_sample[2].split('/')[-1]
                loaded_center_image = imgio.imread(name_center)
                loaded_left_image = imgio.imread(name_left)
                loaded_right_image = imgio.imread(name_right)
                center_image = loaded_center_image
                center_angle = float(batch_sample[3])
                left_image = loaded_left_image
                left_angle = float(batch_sample[3]) + steering_correction
                right_image = loaded_right_image
                right_angle = float(batch_sample[3]) - steering_correction
                # Convert image to the YUV color scheme
                #center_image = image_manipulation(center_image)
                images.append(center_image)
                images.append(left_image)
                images.append(right_image)
                angles.append(center_angle)
                angles.append(left_angle)
                angles.append(right_angle)
            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield utls.shuffle(X_train, y_train)

def image_manipulation(in_image):
    dim = (200, 66)
    resized_img = cv2.resize(in_image, dim, interpolation = cv2.INTER_AREA)
    image_yuv = cv2.cvtColor(resized_img, cv2.COLOR_RGB2YUV)
    #Y_img, U_img, V_img = cv2.split(image_yuv)
    #out_image = np.stack([Y_s, U_i, V_i])
    return image_yuv

batch_size = 32
steering_correction = 0.05
# Combine the driving data from 2 sources
samples_new = []
samples_first_source = merge_driving_data(path_data1, samples_new)
samples_final = merge_driving_data(path_data2, samples_first_source)
# Combine the images from 2 sources
copy_img(path_img1, path_img2)
        
# Split the training and validation data
train_samples, validation_samples = train_test_split(samples_final, test_size=0.2)

# compile and train the model using the generator function
train_generator = generator(train_samples, path_img1, steering_correction, batch_size=batch_size)
validation_generator = generator(validation_samples, path_img1, steering_correction, batch_size=batch_size)

ch, row, col = 3, 160, 320
cropping_rows_top = 50
cropping_rows_bottom = 20
ch_cropped, row_cropped, col_cropped = ch, row-(cropping_rows_top+cropping_rows_bottom), col

## Create a CNN model. We are taking the Nvidia's End-to-End Deep Learning model
model = Sequential()
# Cropping
model.add(Cropping2D(cropping=((cropping_rows_top, cropping_rows_bottom), (0,0)), input_shape=(row,col,ch)))
# Normalization
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(row_cropped,col_cropped,ch_cropped), output_shape=(row_cropped,col_cropped,ch_cropped)))
# Convolutional layer 1
model.add(Conv2D(24, (5,5), subsample=(2,2), activation = 'relu', padding='valid'))
# Convolutional layer 2
model.add(Conv2D(36, (5,5), subsample=(2,2), activation = 'relu', padding='valid'))
# Convolutional layer 3
model.add(Conv2D(48, (5,5), subsample=(2,2), activation = 'relu', padding='valid'))
# Convolutional layer 4
model.add(Conv2D(64, (3,3), activation = 'relu'))
# Convolutional layer 5
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(Flatten())
# Dropout layer
model.add(Dropout(0.5))
# Fully connected layers
model.add(Dense(1164))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, \
            steps_per_epoch=np.ceil(len(train_samples)/batch_size), \
            validation_data=validation_generator, \
            validation_steps=np.ceil(len(validation_samples)/batch_size), \
            epochs=5, verbose=1)

model.save('model.h5')

[['IMG/center_2016_12_01_13_30_48_287.jpg', ' IMG/left_2016_12_01_13_30_48_287.jpg', ' IMG/right_2016_12_01_13_30_48_287.jpg', '0', '0', '0', ' 22.14829'], ['IMG/center_2016_12_01_13_30_48_404.jpg', ' IMG/left_2016_12_01_13_30_48_404.jpg', ' IMG/right_2016_12_01_13_30_48_404.jpg', '0', '0', '0', ' 21.87963'], ['IMG/center_2016_12_01_13_31_13_037.jpg', ' IMG/left_2016_12_01_13_31_13_037.jpg', ' IMG/right_2016_12_01_13_31_13_037.jpg', '0', '0', '0', ' 1.438419'], ['IMG/center_2016_12_01_13_31_13_177.jpg', ' IMG/left_2016_12_01_13_31_13_177.jpg', ' IMG/right_2016_12_01_13_31_13_177.jpg', '0', '0', '0', ' 1.418236'], ['IMG/center_2016_12_01_13_31_13_279.jpg', ' IMG/left_2016_12_01_13_31_13_279.jpg', ' IMG/right_2016_12_01_13_31_13_279.jpg', '0', '0', '0', ' 1.403993'], ['IMG/center_2016_12_01_13_31_13_381.jpg', ' IMG/left_2016_12_01_13_31_13_381.jpg', ' IMG/right_2016_12_01_13_31_13_381.jpg', '0', '0', '0', ' 1.389892'], ['IMG/center_2016_12_01_13_31_13_584.jpg', ' IMG/left_2016_12_01_13_3

"path_img = 'IMG1/'\npath_data = 'driving_log.csv'\n\nfrom keras.models import Sequential\nfrom keras.layers import Lambda\nfrom keras.layers import Dense, Flatten, Conv2D, Dropout, Cropping2D\nfrom sklearn.model_selection import train_test_split\nimport sklearn.utils as utls\nimport imageio as imgio\nimport numpy as np\nimport cv2\nimport csv\n\nsamples = []\nwith open(path_data) as csvfile:\n    reader = csv.reader(csvfile)\n    for line in reader:\n        if line[0] == 'center':\n            continue\n        samples.append(line)\n\nsteering_correction = 0.2\ntrain_samples, validation_samples = train_test_split(samples, test_size=0.2)\n\ndef generator(samples, path_image, steering_correction, batch_size=32):\n    num_samples = len(samples)\n    while 1: # Loop forever so the generator never terminates\n        utls.shuffle(samples)\n        for offset in range(0, num_samples, batch_size):\n            batch_samples = samples[offset:offset+batch_size]\n            images = []\n     